# MRKL

This notebook showcases using an agent to replicate the MRKL chain.

This uses the example Chinook database.
To set it up follow the instructions on https://database.guide/2-sample-databases-sqlite/, placing the `.db` file in a notebooks folder at the root of this repository.

In [8]:
from langchain import LLMMathChain, OpenAI, SerpAPIChain, SQLDatabase, SQLDatabaseChain
from langchain.agents import initialize_agent, Tool

In [9]:
llm = OpenAI(temperature=0)
search = SerpAPIChain()
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
db = SQLDatabase.from_uri("sqlite:///../../../notebooks/Chinook.db")
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)
tools = [
#     Tool(
#         name = "Search",
#         func=search.run,
#         description="useful for when you need to answer questions about current events"
#     ),
#     Tool(
#         name="Calculator",
#         func=llm_math_chain.run,
#         description="useful for when you need to answer questions about math"
#     ),
    Tool(
        name="FooBar DB",
        func=db_chain.run,
        description="useful for when you need to answer questions about FooBar. Input should be in the form of a question"
    )
]

In [10]:
mrkl = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [11]:
mrkl.run("What was our total number of sales and total revenue in 2013?")



> Entering new chain...
What was our total number of sales and total revenue in 2013?
Thought: I need to find out how many sales and how much revenue we had in 2013
Action: FooBar DB
Action Input: sales 2013

> Entering new chain...
sales 2013
SQLQuery: SELECT SUM(Total) FROM Invoice WHERE InvoiceDate BETWEEN '2013-01-01' AND '2013-12-31'
SQLResult: [(450.58000000000027,)]
Answer: 450.58
> Finished chain.

Observation:  450.58
Thought: I need to find out how many sales and how much revenue we had in 2013
Action: FooBar DB
Action Input: revenue 2013

> Entering new chain...
revenue 2013
SQLQuery: SELECT SUM(Total) FROM Invoice WHERE strftime('%Y', InvoiceDate) = '2013'
SQLResult: [(450.58000000000027,)]
Answer: 450.58
> Finished chain.

Observation:  450.58
Thought: I now know the final answer
Final Answer: We had 450.58 sales and 450.58 revenue in 2013
> Finished chain.


'We had 450.58 sales and 450.58 revenue in 2013'

In [18]:
from langchain.agents import ZeroShotAgent
from langchain import PromptTemplate
from langchain import LLMChain
TEMPLATE = """You are a data engineer answering questions using a SQL database.

Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. If a previous query produced an error, do NOT try it again.

Only use the following tables:

{table_info}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: SQLDB
Action Input: the query to run against the SQL database
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {{input}}""".format(**{
            "dialect": db.dialect,
            "table_info": db.table_info})
prompt = PromptTemplate(template=TEMPLATE, input_variables=["input"])

In [19]:
llm_chain = LLMChain(llm=llm, prompt=prompt)
tools = [
    Tool("SQLDB", db.run, "foo")
]
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)

In [20]:
agent.run("What was our total number of sales and total revenue in 2013?")



> Entering new chain...
What was our total number of sales and total revenue in 2013?
Thought: I need to find the total number of sales and total revenue
Action: SQLDB
Action Input: SELECT SUM(Total) AS "Total Sales", SUM(Total) AS "Total Revenue" FROM Invoice WHERE InvoiceDate LIKE '2013%'
Observation: [(450.58000000000027, 450.58000000000027)]
Thought: I now know the final answer
Final Answer: 450.58
> Finished chain.


'450.58'

In [4]:
mrkl.run("What is the age of Olivia Wilde's boyfriend raised to the 0.23 power?")

What is the age of Olivia Wilde's boyfriend raised to the 0.23 power?
Thought: I need to find the age of Olivia Wilde's boyfriend
Action: Search
Action Input: "Olivia Wilde's boyfriend"
Observation: Olivia Wilde started dating Harry Styles after ending her years-long engagement to Jason Sudeikis — see their relationship timeline.
Thought: I need to find the age of Harry Styles
Action: Search
Action Input: "Harry Styles age"
Observation: 28 years
Thought: I need to calculate 28 to the 0.23 power
Action: Calculator
Action Input: 28^0.23

> Entering new chain...
28^0.23

```python
print(28**0.23)
```

Answer: 2.1520202182226886

> Finished chain.

Observation: Answer: 2.1520202182226886

Thought: I now know the final answer
Final Answer: 2.1520202182226886

'2.1520202182226886'

In [5]:
mrkl.run("Who recently released an album called 'The Storm Before the Calm' and are they in the FooBar database? If so, what albums of theirs are in the FooBar database?")

Who recently released an album called 'The Storm Before the Calm' and are they in the FooBar database? If so, what albums of theirs are in the FooBar database?
Thought: I need to find an album called 'The Storm Before the Calm'
Action: Search
Action Input: "The Storm Before the Calm album"
Observation: The Storm Before the Calm (stylized in all lowercase) is the tenth (and eighth international) studio album by Canadian-American singer-songwriter Alanis ...
Thought: I need to check if Alanis is in the FooBar database
Action: FooBar DB
Action Input: "Does Alanis Morissette exist in the FooBar database?"

> Entering new chain...
Does Alanis Morissette exist in the FooBar database?
SQLQuery: SELECT * FROM Artist WHERE Name = 'Alanis Morissette'
SQLResult: [(4, 'Alanis Morissette')]
Answer: Yes
> Finished chain.

Observation:  Yes
Thought: I need to find out what albums of Alanis's are in the FooBar database
Action: FooBar DB
Action Input: "What albums by Alanis Morissette are in the FooBar

'The album is by Alanis Morissette and the albums in the FooBar database by her are Jagged Little Pill'